In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import Packages.ClusteringHelper as ch
import numpy as np
from tqdm.notebook import tqdm
from collections import Counter
from Packages.TimeEvolving import Cluster

In [6]:
text, data = ch.read_aida_yago_conll(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv")
save = False
if save:
    text_file = open('text.txt', 'w')
    text_file.write(text)
    text_file.close()
data

,documents,tokens,indexes,word_indexes,mentions,entities,wikidatas,numeric_codes,alpha_codes
0,0,EU,"(0, 2)",0,,,,,
1,0,rejects,"(3, 10)",1,,,,,
2,0,German,"(11, 17)",2,German,Germany,http://en.wikipedia.org/wiki/Germany,11867,/m/0345h
3,0,call,"(18, 22)",3,,,,,
4,0,to,"(23, 25)",4,,,,,
...,...,...,...,...,...,...,...,...,...
285584,1392,younger,"(1342, 1349)",265,,,,,
285585,1392,brother,"(1350, 1357)",266,,,,,
285586,1392,",","(1358, 1359)",267,,,,,
285587,1392,Bobby,"(1360, 1365)",268,Bobby,Bobby_Charlton,http://en.wikipedia.org/wiki/Bobby_Charlton,4224,/m/01c8x


In [7]:
ents_data = data[data['entities'] != ''].copy()

In [8]:
from pathlib import Path

path_train = Path(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\encodings\\AIDA-YAGO2_train_encodings.jsonl")
path_testa = Path(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\encodings\\AIDA-YAGO2_testa_encodings.jsonl")
path_testb = Path(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\encodings\\AIDA-YAGO2_testb_encodings.jsonl")
raw_encodings_train = open(path_train, 'r').read()
raw_encodings_testa = open(path_testa, 'r').read()
raw_encodings_testb = open(path_testb, 'r').read()

In [9]:
import json

jsonl_parsed_train = [json.loads(x) for x in raw_encodings_train.splitlines()]
jsonl_parsed_testa = [json.loads(x) for x in raw_encodings_testa.splitlines()]
jsonl_parsed_testb = [json.loads(x) for x in raw_encodings_testb.splitlines()]
jsonl_parsed = jsonl_parsed_train + jsonl_parsed_testa + jsonl_parsed_testb

In [10]:
encodings = [x['encoding'] for x in jsonl_parsed]

In [11]:
ents_data['encodings'] = encodings
# ents_data_filtered = ch.filter_data(ents_data, 3)
ents_data_filtered = ents_data.copy()

In [12]:
ents_data = ents_data_filtered[ents_data_filtered['entities'] != '']
entities = ents_data['entities'].values
mentions = ents_data['tokens'].values
encodings = ents_data['encodings'].values
unique_entities = np.unique(entities)
gold_standard = Counter(entities)
gold_standard = dict(sorted(gold_standard.items(), key=lambda item: item[1], reverse=True))

In [13]:
leven_cluster = np.loadtxt("../aida-yago2-dataset/damerau_1_2.txt", dtype=np.int32)
lev_cluster_dict = {}
for i, x in enumerate(leven_cluster):
    try:
        lev_cluster_dict[x].append(
            (mentions[i], entities[i], encodings[i]))
    except:
        lev_cluster_dict[x] = [
            (mentions[i], entities[i], encodings[i])]

In [14]:
cluster_all = []
from sklearn.cluster import AgglomerativeClustering
clusterizator = AgglomerativeClustering(n_clusters=None, affinity='cosine',
                                        distance_threshold=0.03678974213038678, linkage="single")

for x in lev_cluster_dict:
    encodings = [y[2] for y in lev_cluster_dict[x]]
    if len(encodings) > 1:
        cluster = clusterizator.fit_predict(encodings)
        cluster_all.append(cluster)
    else:
        cluster_all.append(np.zeros(1))

In [15]:
total_encodings = [[y[2] for y in lev_cluster_dict[x]] for x in lev_cluster_dict.keys()]
clustered_entities = [[row[1] for row in lev_cluster_dict[index]] for index in lev_cluster_dict.keys()]
clustered_mentions = [[row[0] for row in lev_cluster_dict[index]] for index in lev_cluster_dict.keys()]

In [16]:
# clusters_splitted = []
# clusters_vectors = []
# for n_cluster in range(len(clustered_entities)):
#     cluster_unique = dict.fromkeys(set(cluster_all[n_cluster]), [])
#     cluster_unique_vector = dict.fromkeys(set(cluster_all[n_cluster]), [])
#     for key in cluster_unique:
#         cluster_unique[key] = [clustered_entities[n_cluster][index] for index in
#                                range(len(clustered_entities[n_cluster])) if cluster_all[n_cluster][index] == key]
#         cluster_unique_vector[key] = [total_encodings[n_cluster][index] for index in
#                                range(len(clustered_entities[n_cluster])) if cluster_all[n_cluster][index] == key]
#     clusters_vectors.append(cluster_unique_vector)
#     clusters_splitted.append(cluster_unique)

In [17]:
clusters_splitted = []
clusters_vectors = []
for n_cluster in range(len(clustered_entities)):
    cluster_unique = {k:Cluster() for k in set(cluster_all[n_cluster])}
    for key in cluster_unique:
        cl_mentions = [clustered_mentions[n_cluster][index] for index in
                               range(len(clustered_mentions[n_cluster])) if cluster_all[n_cluster][index] == key]
        cl_entities = [clustered_entities[n_cluster][index] for index in
                               range(len(clustered_entities[n_cluster])) if cluster_all[n_cluster][index] == key]
        cl_encodings = [total_encodings[n_cluster][index] for index in
                               range(len(clustered_entities[n_cluster])) if cluster_all[n_cluster][index] == key]
        cluster_unique[key] = Cluster(entities=cl_entities, mentions=cl_mentions,encodings_list=cl_encodings)
    clusters_splitted.append(cluster_unique)

In [18]:
# clusters_splitted_vector_dict = [np.mean(x[y], axis=0)  for x in clusters_vectors for y in x]
# clusters_splitted_vector_dict = [np.median(x[y], axis=0)  for x in clusters_vectors for y in x]

In [19]:
clusters_splitted_dict = [x[y] for x in clusters_splitted for y in x]

In [20]:
clusterizator = AgglomerativeClustering(n_clusters=None, affinity='cosine',
                                        distance_threshold=0.0155, linkage="single")

clusters_of_cluster = clusterizator.fit_predict([x.encodings_mean() for x in clusters_splitted_dict])
len(set(clusters_of_cluster))

6244

In [21]:
coc_dict = {k: Cluster() for k in set(clusters_of_cluster)}
for index, el in enumerate(clusters_of_cluster):
    coc_dict[el] = coc_dict[el] + clusters_splitted_dict[index]
len(coc_dict.keys())

6244

In [22]:
import Packages.ClusteringHelper as ch
golden_standard_dict = ch.get_gold_standard_dict(ents_data)

### B-CUBED

In [23]:
list_of_counter = [x.count_ents() for x in coc_dict.values()]
print('Counter finito')
from tqdm.notebook import tqdm
# B-cubed
bcubed_precision_num = 0
bcubed_recall_num = 0
for cluster in tqdm(list_of_counter):
    for gold_key in golden_standard_dict.keys():
        try:
            bcubed_precision_num = bcubed_precision_num + (pow(cluster[gold_key], 2) /
                                       sum(cluster.values()))
            bcubed_recall_num = bcubed_recall_num + (pow(cluster[gold_key], 2) /
                                       golden_standard_dict[gold_key])
        except:
            pass

bcubed_precision = bcubed_precision_num/sum([x.n_elements() for x in coc_dict.values()])
bcubed_recall = bcubed_recall_num/ents_data.shape[0]

bcubed_precision

Counter finito


  0%|          | 0/6244 [00:00<?, ?it/s]

0.8894849007000544

In [24]:
bcubed_recall

0.8737135964453578

In [25]:
bcubed_f1 = (2 * (bcubed_recall * bcubed_precision)) / (
        bcubed_precision + bcubed_recall)
bcubed_f1

0.8815287136787912

CEAFm precision

In [26]:
best_alignment = ch.get_optimal_alignment([x.count_ents() for x in coc_dict.values()], set(entities),
                                          is_dict=False)

In [27]:
# CEAFm_precision
CEAFm_p = sum(best_alignment.values()) / len(entities)
CEAFm_p

0.8465686450731567

CEAFm recall

In [28]:
# CEAFm_recall
CEAFm_r = sum(best_alignment.values()) / sum([x.n_elements() for x in coc_dict.values()])
CEAFm_r

0.8465686450731567

CEAFm Fscore

In [29]:
# CEAFm_f1
CEAFm_f1 = 2 * (CEAFm_p * CEAFm_r) / (CEAFm_p + CEAFm_r)
CEAFm_f1

0.8465686450731568

In [30]:
to_print = True
if to_print:
    import sys
    original_stdout = sys.stdout
    with open(".\\full_approach.html", "a", encoding='utf-8') as f:
        sys.stdout = f
        print('<html>')
        print("CEAFm-R:", CEAFm_r)
        print("CEAFm-P:", CEAFm_p)
        print("CEAFm:", CEAFm_f1)
        print("<br>")
        print("bcubed_recall:", bcubed_recall)
        print("bcubed_precision:", bcubed_precision)
        print("bcubed_f1:", bcubed_f1)
        print("<br>", "Clusters:", '<br>')
        print("<br> Clusters:", '<br>')
        print(*list(coc_dict.values()), sep=" <br><br> ")
        print("<br>")
        print("<br>")
        print("Gold_standard:", '<br>')
        print(dict(Counter(entities)))
        print('</html>')
        sys.stdout = original_stdout

  0%|          | 0/6244 [00:00<?, ?it/s]